In [2]:

import argparse
import torch
# import lib
import numpy as np
import os
import datetime
from dataset import *
from loss import *
from model import *
from optimizer import *
from trainer import *
from torch.utils import data
import pickle
import sys
from dataset_time import *
from logger import *

import sys
sys.path.insert(0, '../PyTorch_GBW_LM')
sys.path.insert(0, '../PyTorch_GBW_LM/log_uniform')

from sparse_model import RNNModel, SampledSoftmax

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--hidden_size', default=50, type=int)
parser.add_argument('--num_layers', default=1, type=int)
parser.add_argument('--batch_size', default=100, type=int)
parser.add_argument('--dropout_input', default=0, type=float)
parser.add_argument('--dropout_hidden', default=.2, type=float)

# parse the optimizer arguments
parser.add_argument('--optimizer_type', default='Adagrad', type=str)
parser.add_argument('--final_act', default='tanh', type=str)
parser.add_argument('--lr', default=.05, type=float)
parser.add_argument('--weight_decay', default=0.0, type=float)
parser.add_argument('--momentum', default=0.1, type=float)
parser.add_argument('--eps', default=1e-6, type=float)

parser.add_argument("-seed", type=int, default=7,
					 help="Seed for random initialization")
parser.add_argument("-sigma", type=float, default=None,
					 help="init weight -1: range [-sigma, sigma], -2: range [0, sigma]")
parser.add_argument("--embedding_dim", type=int, default=-1,
					 help="using embedding")
# parse the loss type
parser.add_argument('--loss_type', default='TOP1', type=str)
# parser.add_argument('--loss_type', default='BPR', type=str)
parser.add_argument('--topk', default=5, type=int)
# etc
parser.add_argument('--bptt', default=1, type=int)
parser.add_argument('--test_observed', default=5, type=int)
parser.add_argument('--window_size', default=30, type=int)
parser.add_argument('--warm_start', default=5, type=int)

parser.add_argument('--n_epochs', default=20, type=int)
parser.add_argument('--time_sort', default=False, type=bool)
parser.add_argument('--model_name', default='GRU4REC', type=str)
parser.add_argument('--save_dir', default='models', type=str)
parser.add_argument('--data_folder', default='../Data/movielen/1m/', type=str)
parser.add_argument('--data_action', default='item.pickle', type=str)
parser.add_argument('--data_cate', default='cate.pickle', type=str)
parser.add_argument('--data_time', default='time.pickle', type=str)
parser.add_argument("--is_eval", action='store_true')
parser.add_argument('--load_model', default=None,  type=str)
parser.add_argument('--checkpoint_dir', type=str, default='checkpoint')
parser.add_argument('--data_name', default=None, type=str)
parser.add_argument('--shared_embedding', default=None, type=int)
parser.add_argument('--patience', default=1000)
parser.add_argument('--negative_num', default=1000, type=int)

# Get the arguments
args = parser.parse_args(args=[])
args.cuda = torch.cuda.is_available()


In [4]:
args.data_folder = "../Data/tmall/100k_unknown_cate/"
args.data_action = "cate_time.pickle"
args.data_cate = "cate_time.pickle"
args.data_time = "time_time.pickle"
args.data_name = "taobao"
args.valid_start_time = 1512172800
valid_start_time = args.valid_start_time
args.test_start_time = 1512259200
test_start_time = args.test_start_time

In [11]:
args.embedding_dim = 64
args.hidden_size = 300
args.lr = 0.0001
args.window_size = 5
args.test_observed = 5
args.n_epochs = 300
args.shared_embedding = 1
args.batch_size = 300
args.optimizer_type = "Adam"

In [6]:
data_file = "data.pickle"
f = open(data_file, "rb")
data_map = pickle.load(f)
data_obj = data_map["data"]

In [12]:
hidden_size = args.hidden_size
num_layers = args.num_layers
batch_size = args.batch_size
dropout_input = args.dropout_input
dropout_hidden = args.dropout_hidden
embedding_dim = args.embedding_dim
final_act = args.final_act
loss_type = args.loss_type
topk = args.topk
optimizer_type = args.optimizer_type
lr = args.lr
weight_decay = args.weight_decay
momentum = args.momentum
eps = args.eps
BPTT = args.bptt

n_epochs = args.n_epochs
time_sort = args.time_sort

window_size = args.window_size
shared_embedding = args.shared_embedding

if embedding_dim == -1:
    print("embedding dim not -1", embedding_dim)
    raise AssertionError()

observed_threshold = args.test_observed

data_action = args.data_folder+args.data_action
data_cate = args.data_folder+args.data_cate
data_time = args.data_folder+args.data_time

data_obj = MYDATA(data_action, data_cate, data_time, valid_start_time, test_start_time, observed_threshold, window_size)

train_data = data_obj.train_dataset

print("+"*10)
print("valid load")

valid_data = data_obj.test_dataset
test_data = data_obj.test_dataset

input_size = data_obj.items()
output_size = input_size

negative_num = args.negative_num
# print("input_size", input_size)

# train_data_loader = dataset.DataLoader(train_data, batch_size)

valid_data_loader = dataset.DataLoader(valid_data, batch_size)

data_name = args.data_name



action seq num 51275
time seq num 51275
loading item map
loading item map
observed_threshold 5 5
loading data
valid_start_time 1512172800
test start time 1512259200
seq num for training 2738883
seq num of actions for training 2738883
seq num for testing 430797
seq num of actions for testing 430797
++++++++++
valid load
item num 202
seq num 430797
batch size 300
batch_num 1435


In [13]:
input_size

202

In [14]:
ss = SampledSoftmax(output_size, negative_num, embedding_dim, None)

log = Logger()
log.addIOWriter(args)

model = GRU4REC(log, ss, input_size, hidden_size, output_size,
                    final_act=final_act,
                    num_layers=num_layers,
                    use_cuda=args.cuda,
                    dropout_input=dropout_input,
                    dropout_hidden=dropout_hidden,
                    embedding_dim=embedding_dim, 
                    shared_embedding=shared_embedding
                    )

output_file hcdmg3_23_16_31_05_300_300_300_Adam_0.0001_5_1_taobao
share embedding


In [15]:
save_path = "../log/samplePaddingSessionRNN/checkpoint/01191420/model_best.pt"

save_model_data = torch.load(save_path)
# print(save_model_data)
model.load_state_dict(save_model_data['model']) 
model.eval()

RuntimeError: Error(s) in loading state_dict for GRU4REC:
	Missing key(s) in state_dict: "gru.weight_ih_l0", "gru.weight_hh_l0", "gru.bias_ih_l0", "gru.bias_hh_l0", "m_fc.weight", "m_fc.bias". 
	Unexpected key(s) in state_dict: "m_short_gru.weight_ih_l0", "m_short_gru.weight_hh_l0", "m_short_gru.bias_ih_l0", "m_short_gru.bias_hh_l0". 

In [10]:
model.look_up.weight.data

tensor([[-0.0878,  1.7085, -0.8443,  ...,  1.8302, -0.0360,  0.3190],
        [ 0.0344,  0.1262, -0.1933,  ...,  0.1119, -0.4377, -0.0741],
        [ 0.3102, -0.0353,  0.1554,  ...,  0.2212, -0.4533, -0.0420],
        ...,
        [-0.3312,  1.3374, -0.1737,  ...,  0.8973,  0.8089,  0.0803],
        [ 0.9125, -1.2364, -1.0854,  ...,  0.2383,  0.6634,  0.0425],
        [ 0.0952, -0.0128, -0.2247,  ...,  0.1301,  0.1469,  0.0236]],
       device='cuda:0')

In [11]:
model.m_ss.params.weight.data

tensor([[-0.0878,  1.7085, -0.8443,  ...,  1.8302, -0.0360,  0.3190],
        [ 0.0344,  0.1262, -0.1933,  ...,  0.1119, -0.4377, -0.0741],
        [ 0.3102, -0.0353,  0.1554,  ...,  0.2212, -0.4533, -0.0420],
        ...,
        [-0.3312,  1.3374, -0.1737,  ...,  0.8973,  0.8089,  0.0803],
        [ 0.9125, -1.2364, -1.0854,  ...,  0.2383,  0.6634,  0.0425],
        [ 0.0952, -0.0128, -0.2247,  ...,  0.1301,  0.1469,  0.0236]],
       device='cuda:0')

In [12]:
recall = save_model_data['recall']
print("recall", recall)

mrr = save_model_data['mrr']
print("mrr", mrr)

recall 0.9730034843205575
mrr 0.5649806367603328


In [13]:
loss_type = "XE"
loss_func = LossFunction(loss_type=loss_type, use_cuda=args.cuda)

In [15]:
model.eval()
dataloader = valid_data_loader

losses = []
recalls = []
mrrs = []
weights = []

topk = 1
eval_iter = 0
device = torch.device('cuda')
warm_start = args.warm_start
with torch.no_grad():
    total_test_num = []

    for x_batch, y_batch, x_len_batch, idx_batch in dataloader:        
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        warm_start_mask = (idx_batch>=warm_start)

        hidden = model.init_hidden()

        output_batch = model(x_batch, hidden, x_len_batch)

        # et_1 = datetime.datetime.now()
        # print("duration 1", et_1-st)

        sampled_logit_batch, sampled_target_batch = model.m_ss(output_batch, y_batch)

        loss_batch = loss_func(sampled_logit_batch, sampled_target_batch)
        losses.append(loss_batch.item())

        # et_2 = datetime.datetime.now()
        # print("duration 2", et_2-et_1)

        # logit_batch = self.model.m_ss.params(output_batch)
        recall_batch, mrr_batch = evaluate(sampled_logit_batch, sampled_target_batch, warm_start_mask, k=topk)

        weights.append( int( warm_start_mask.int().sum() ) )
        recalls.append(recall_batch)
        mrrs.append(mrr_batch)

        total_test_num.append(y_batch.view(-1).size(0))

mean_losses = np.mean(losses)
mean_recall = np.average(recalls, weights=weights)
mean_mrr = np.average(mrrs, weights=weights)
print("total_test_num", np.sum(total_test_num))
print("recall", mean_recall)
print("mrr", mean_mrr)

shuffling
batch num 1435
total_test_num 430500
recall 0.4364250871080139
mrr 0.4364250876969992


In [19]:
batch_size = 2
valid_data_loader = dataset.DataLoader(valid_data, batch_size)
model.batch_size = batch_size

In [26]:
model.eval()
dataloader = valid_data_loader

losses = []
recalls = []
mrrs = []
weights = []

topk = 1
eval_iter = 0
device = torch.device('cuda')
warm_start = args.warm_start
with torch.no_grad():
    total_test_num = []

    for x_batch, y_batch, x_len_batch, idx_batch in dataloader:
        print("*"*10)
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        warm_start_mask = (idx_batch>=warm_start)

        hidden = model.init_hidden()

        output_batch = model(x_batch, hidden, x_len_batch)

        # et_1 = datetime.datetime.now()
        # print("duration 1", et_1-st)

        sampled_logit_batch, sampled_target_batch = model.m_ss(output_batch, y_batch)
        
        pred_cate_logit, pred_cate_index = sampled_logit_batch.topk(1, dim=-1)
        
        y_batch = y_batch
        
        if pred_cate_index[0] == y_batch[0]:
            print("0 pred", pred_cate_index[0])
            print("0 target", y_batch[0])
            print("0 x batch", x_batch[0])
            
        if pred_cate_index[1] == y_batch[1]:
            print("1 pred", pred_cate_index[1])
            print("1 target", y_batch[1])
            print("1 x batch", x_batch[1])
#         loss_batch = loss_func(sampled_logit_batch, sampled_target_batch)
#         losses.append(loss_batch.item())

#         # et_2 = datetime.datetime.now()
#         # print("duration 2", et_2-et_1)

#         # logit_batch = self.model.m_ss.params(output_batch)
#         recall_batch, mrr_batch = evaluate(sampled_logit_batch, sampled_target_batch, warm_start_mask, k=topk)

#         weights.append( int( warm_start_mask.int().sum() ) )
#         recalls.append(recall_batch)
#         mrrs.append(mrr_batch)

        total_test_num.append(y_batch.view(-1).size(0))

mean_losses = np.mean(losses)
mean_recall = np.average(recalls, weights=weights)
mean_mrr = np.average(mrrs, weights=weights)
print("total_test_num", np.sum(total_test_num))
print("recall", mean_recall)
print("mrr", mean_mrr)

shuffling
batch num 215398
**********
**********
0 pred tensor([177], device='cuda:0')
0 target tensor(177, device='cuda:0')
0 x batch tensor([  6,   1,   3,   1, 177], device='cuda:0')
1 pred tensor([201], device='cuda:0')
1 target tensor(201, device='cuda:0')
1 x batch tensor([ 2, 43, 68, 43, 77], device='cuda:0')
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([201, 201, 201, 201,  42], device='cuda:0')
**********
0 pred tensor([83], device='cuda:0')
0 target tensor(83, device='cuda:0')
0 x batch tensor([83, 83, 83, 83, 83], device='cuda:0')
1 pred tensor([2], device='cuda:0')
1 target tensor(2, device='cuda:0')
1 x batch tensor([22,  2,  2,  2,  2], device='cuda:0')
**********
0 pred tensor([2], device='cuda:0')
0 target tensor(2, device='cuda:0')
0 x batch tensor([7, 9, 4, 7, 2], device='cuda:0')
**********
**********
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([201,

1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([ 1,  1,  6, 69,  1], device='cuda:0')
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([201, 201, 201, 201, 201], device='cuda:0')
**********
0 pred tensor([8], device='cuda:0')
0 target tensor(8, device='cuda:0')
0 x batch tensor([ 8,  8,  2, 68,  8], device='cuda:0')
**********
1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([4, 1, 1, 1, 1], device='cuda:0')
**********
1 pred tensor([6], device='cuda:0')
1 target tensor(6, device='cuda:0')
1 x batch tensor([20, 20,  1,  6,  6], device='cuda:0')
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([ 40, 201, 201, 201, 201], device='cuda:0')
**********
1 pred tensor([8], device='cuda:0')
1 target tensor(8, device='cuda:0')
1 x batch tensor([8, 8, 8, 8, 8], device='cuda:0')
**********
0 pred tensor([201]

0 target tensor(87, device='cuda:0')
0 x batch tensor([201,  87,  87,  87,  87], device='cuda:0')
1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([4, 6, 6, 1, 1], device='cuda:0')
**********
**********
1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([1, 1, 1, 1, 1], device='cuda:0')
**********
0 pred tensor([141], device='cuda:0')
0 target tensor(141, device='cuda:0')
0 x batch tensor([141, 141, 141, 141, 141], device='cuda:0')
1 pred tensor([17], device='cuda:0')
1 target tensor(17, device='cuda:0')
1 x batch tensor([ 1,  4,  1, 17, 17], device='cuda:0')
**********
1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([ 6,  3, 21, 10,  1], device='cuda:0')
**********
0 pred tensor([1], device='cuda:0')
0 target tensor(1, device='cuda:0')
0 x batch tensor([6, 1, 6, 1, 1], device='cuda:0')
**********
**********
0 pred tensor([2], device='cuda:0')
0 target tensor(2, device

0 pred tensor([35], device='cuda:0')
0 target tensor(35, device='cuda:0')
0 x batch tensor([  6, 139, 139,  35,  35], device='cuda:0')
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([201, 201, 201, 201, 201], device='cuda:0')
**********
1 pred tensor([4], device='cuda:0')
1 target tensor(4, device='cuda:0')
1 x batch tensor([4, 4, 4, 4, 4], device='cuda:0')
**********
0 pred tensor([58], device='cuda:0')
0 target tensor(58, device='cuda:0')
0 x batch tensor([58, 58, 58, 58, 58], device='cuda:0')
**********
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([201, 201, 201, 201, 201], device='cuda:0')
1 pred tensor([25], device='cuda:0')
1 target tensor(25, device='cuda:0')
1 x batch tensor([45, 69, 69, 25, 25], device='cuda:0')
**********
0 pred tensor([3], device='cuda:0')
0 target tensor(3, device='cuda:0')
0 x batch tensor([21,  3, 13,  3,  3], device='cuda:0')
**********
0 p

1 pred tensor([201], device='cuda:0')
1 target tensor(201, device='cuda:0')
1 x batch tensor([ 63,  63,  47, 201, 201], device='cuda:0')
**********
1 pred tensor([9], device='cuda:0')
1 target tensor(9, device='cuda:0')
1 x batch tensor([2, 9, 9, 2, 9], device='cuda:0')
**********
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([201, 201, 201, 201, 201], device='cuda:0')
1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([6, 1, 1, 1, 1], device='cuda:0')
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([106,  60,  64, 201, 201], device='cuda:0')
1 pred tensor([9], device='cuda:0')
1 target tensor(9, device='cuda:0')
1 x batch tensor([ 6,  1,  9, 10,  9], device='cuda:0')
**********
0 pred tensor([5], device='cuda:0')
0 target tensor(5, device='cuda:0')
0 x batch tensor([201, 201,   5,   5,   5], device='cuda:0')
1 pred tensor([6], device='

...

1 pred tensor([201], device='cuda:0')
1 target tensor(201, device='cuda:0')
1 x batch tensor([  2, 201, 201, 201, 201], device='cuda:0')
**********
1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([ 1,  1,  1,  1, 53], device='cuda:0')
**********
0 pred tensor([104], device='cuda:0')
0 target tensor(104, device='cuda:0')
0 x batch tensor([104, 201, 201, 104, 104], device='cuda:0')
**********
1 pred tensor([201], device='cuda:0')
1 target tensor(201, device='cuda:0')
1 x batch tensor([  7,   7, 120,   8, 201], device='cuda:0')
**********
1 pred tensor([2], device='cuda:0')
1 target tensor(2, device='cuda:0')
1 x batch tensor([2, 2, 2, 2, 2], device='cuda:0')
**********
1 pred tensor([3], device='cuda:0')
1 target tensor(3, device='cuda:0')
1 x batch tensor([ 8,  3,  5,  3, 48], device='cuda:0')
**********
0 pred tensor([2], device='cuda:0')
0 target tensor(2, device='cuda:0')
0 x batch tensor([5, 5, 2, 2, 2], device='cuda:0')
1 pred tensor([201], 

1 x batch tensor([40,  5,  5, 10, 10], device='cuda:0')
**********
0 pred tensor([2], device='cuda:0')
0 target tensor(2, device='cuda:0')
0 x batch tensor([36,  6, 15,  2,  2], device='cuda:0')
1 pred tensor([33], device='cuda:0')
1 target tensor(33, device='cuda:0')
1 x batch tensor([ 1,  1,  1, 33, 33], device='cuda:0')
**********
**********
0 pred tensor([3], device='cuda:0')
0 target tensor(3, device='cuda:0')
0 x batch tensor([3, 3, 3, 3, 3], device='cuda:0')
1 pred tensor([2], device='cuda:0')
1 target tensor(2, device='cuda:0')
1 x batch tensor([2, 2, 2, 2, 2], device='cuda:0')
**********
**********
**********
1 pred tensor([6], device='cuda:0')
1 target tensor(6, device='cuda:0')
1 x batch tensor([4, 6, 6, 6, 1], device='cuda:0')
**********
**********
1 pred tensor([201], device='cuda:0')
1 target tensor(201, device='cuda:0')
1 x batch tensor([145, 150, 201, 145, 201], device='cuda:0')
**********
0 pred tensor([46], device='cuda:0')
0 target tensor(46, device='cuda:0')
0 x bat

0 target tensor(5, device='cuda:0')
0 x batch tensor([5, 5, 5, 5, 5], device='cuda:0')
1 pred tensor([4], device='cuda:0')
1 target tensor(4, device='cuda:0')
1 x batch tensor([4, 4, 4, 4, 4], device='cuda:0')
**********
0 pred tensor([3], device='cuda:0')
0 target tensor(3, device='cuda:0')
0 x batch tensor([21,  3,  4,  4,  3], device='cuda:0')
**********
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([107, 107,  22, 201, 201], device='cuda:0')
1 pred tensor([4], device='cuda:0')
1 target tensor(4, device='cuda:0')
1 x batch tensor([4, 4, 4, 4, 3], device='cuda:0')
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch tensor([201, 201, 201, 201, 201], device='cuda:0')
1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([1, 1, 1, 1, 1], device='cuda:0')
**********
0 pred tensor([201], device='cuda:0')
0 target tensor(201, device='cuda:0')
0 x batch t

1 pred tensor([1], device='cuda:0')
1 target tensor(1, device='cuda:0')
1 x batch tensor([  1, 201,  74,   1,   1], device='cuda:0')
**********
**********
0 pred tensor([146], device='cuda:0')
0 target tensor(146, device='cuda:0')
0 x batch tensor([188, 188, 201, 146, 146], device='cuda:0')
**********
1 pred tensor([10], device='cuda:0')
1 target tensor(10, device='cuda:0')
1 x batch tensor([ 7, 10, 10, 10, 10], device='cuda:0')
**********
1 pred tensor([17], device='cuda:0')
1 target tensor(17, device='cuda:0')
1 x batch tensor([ 2,  2,  2,  2, 17], device='cuda:0')
**********
0 pred tensor([97], device='cuda:0')
0 target tensor(97, device='cuda:0')
0 x batch tensor([201, 201,   9,  97,  97], device='cuda:0')
1 pred tensor([6], device='cuda:0')
1 target tensor(6, device='cuda:0')
1 x batch tensor([6, 6, 6, 1, 6], device='cuda:0')
**********
1 pred tensor([4], device='cuda:0')
1 target tensor(4, device='cuda:0')
1 x batch tensor([4, 4, 4, 4, 4], device='cuda:0')
**********
0 pred tenso

KeyboardInterrupt: 

In [13]:
model.eval()
dataloader = valid_data_loader

losses = []
recalls = []
mrrs = []
weights = []

nonzero_cate_num_total = 0

topk = 5
eval_iter = 0
device = torch.device('cuda')
warm_start = args.warm_start
with torch.no_grad():
    total_test_num = []

    for x_cate_batch, mask_cate, max_acticonNum_cate, seqLen_cate, x_batch, mask_batch, seqLen_batch, y_batch, idx_batch in dataloader:        
        x_cate_batch = x_cate_batch.to(device)
        mask_cate = mask_cate.to(device)
        
        x_batch = x_batch.to(device)
        mask_batch = mask_batch.to(device)

        y_batch = y_batch.to(device)
        warm_start_mask = (idx_batch>=warm_start).to(device)
        
        nonzero_cate_num = np.count_nonzero(seqLen_cate)
        nonzero_cate_num_total += nonzero_cate_num
#         print("zero_cate_num", zero_cate_num)
#         print("seqLen_cate", seqLen_cate)
#         print(seqLen_cate.nonzero())

#         exit()
print("nonzero_cate_num_total", nonzero_cate_num_total)


batch_num 4158
zero_cate_num_total 634331


In [14]:
634331.0/831600

0.7627837902837903

In [4]:
np.count_nonzero(np.eye(4))

4